# Demo of the Pyrolysis Modeling and Kinetics Computation Capabilities in FireSciPy

This Jupyter notebook demonstrates the modeling and computation capabilites of pyrolysis kinetics in FireSciPy. It contains:

1. A basic example modeling a single-step Arrhenius reaction under a linear temperature program.
2. A more advanced example where synthetic thermogravimetric data is generated, and activation energy ($E$) is estimated using the Kissinger–Akahira–Sunose (KAS) method.
3. Estimation of the apparent activation energy ($E_a$) from experimental TGA data using KAS.

These examples show how FireSciPy can be used both for forward modeling and inverse parameter extraction.


In [ ]:
import os
import re
import sys
import matplotlib

import numpy as np
import scipy as sp
import pandas as pd
import firescipy as fsp
import matplotlib.pyplot as plt

from importlib import reload  # Python 3.4+
from scipy.ndimage import uniform_filter1d


In [ ]:
# Order of plot colors
plt_colors = ["tab:blue", "tab:orange", "tab:green", 
              "tab:red", "tab:purple", "tab:brown", 
              "tab:pink", "tab:gray", "tab:olive", 
              "tab:cyan"]

# Global settings for plotting
plt.rcParams.update({
    'axes.axisbelow': True,     # Keep grid behind plots
    'figure.autolayout': True,  # Equivalent to calling tight_layout()
    'axes.facecolor': 'white',  # Prevents transparent background
    'grid.alpha': 0.6,          # Makes gridlines more readable
    'font.size': 12             # Set global font size
})


In [ ]:
###########################################################################
## ! Use the 'requirements.txt' to create a virtual Python environment ! ##
###########################################################################

# Package Versions
# ----------------
# Python version: 3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]
# Python path: F:\PhD\PythonPackage\venv_Packaging\Scripts\python.exe
# Numpy version: 2.3.2
# SciPy version: 1.16.1
# Pandas version: 2.3.1
# Matplotlib version: 3.10.5
# FireSciPy version: 0.0.5


print('Package Versions')
print('----------------')
print('Python version: {}'.format(sys.version))
print('Python path: {}'.format(sys.executable))
print('Numpy version: {}'.format(np.__version__))
print('SciPy version: {}'.format(sp.__version__))
print('Pandas version: {}'.format(pd.__version__))
print('Matplotlib version: {}'.format(matplotlib.__version__))
print('FireSciPy version: {}'.format(fsp.__version__))


## Single-Step Pyrolysis Kinetics Modelling

In this section, the basic pyrolysis modelling capabilities of FireSciPy are introduced.


### Theoretical Background
----------------------

Pyrolysis kinetics describe the rates of thermally induced decomposition processes and are commonly studied using thermal analysis techniques. Common methods are thermogravimetric analysis (TGA), Differential Scanning Calorimetry (DSC) or Microscale Combustion Calorimetry (MCC), to name a few. The process rates can be described in terms of the temperature $T$, the extent of conversion $\alpha$ and the pressure $P$, see formula (1) below:

<!-- formula 1 -->
$$
\frac{d \alpha}{dt} = k(T) ~f(\alpha) ~h(P)
$$

Where $\frac{d \alpha}{dt}$ is the conversion rate, $k(T)$ is the reaction rate constant, $f(\alpha)$ is the reaction model and $h(P)$ is the pressure dependence. The pressure dependence plays a role when gases can react with the decomposing sample material. During TGA experiments this can be minimized by maintaining an inert atmosphere and ensuring a sufficiently high purge gas flow to swiftly remove evolved gases which might otherwise react with the remaining sample material. Under these conditions the pressure term can than be dropped.

Typically, the reaction rate constant $k(T)$ is expressed as an Arrhenius equation, see formula (2):

<!-- formula 2 -->
$$
k(T) = A ~exp \left( -\frac{E}{R ~T}\right)
$$

Where $A$ is the pre-exponential factor, $E$ the activation energy and $R$ the gas constant.

Substituting equation (2) into (1) and neglecting $h(P)$ leads to equation (3):

<!-- formula 3 -->
$$
\frac{d \alpha}{dt} = A ~exp \left( -\frac{E}{R ~T}\right) ~f(\alpha)
$$

Together, $A$, $E$ and $f(\alpha)$ form the kinetic triplet.


More details are available in the recommendations provided by the International Confederation for Thermal Analysis and Calorimetry (ICTAC) Kinetics Committee:

- [ICTAC Kinetics Committee recommendations for performing kinetic computations on thermal analysis data, 2011 (https://doi.org/10.1016/j.tca.2011.03.034)](https://doi.org/10.1016/j.tca.2011.03.034)
- [ICTAC Kinetics Committee recommendations for collecting experimental thermal analysis data for kinetic computations, 2014 (https://doi.org/10.1016/j.tca.2014.05.036)](https://doi.org/10.1016/j.tca.2014.05.036)
- [ICTAC Kinetics Committee recommendations for analysis of multi-step kinetics, 2020 (https://doi.org/10.1016/j.tca.2020.178597)](https://doi.org/10.1016/j.tca.2020.178597)
- [ICTAC Kinetics Committee recommendations for analysis of thermal decomposition kinetics, 2023 (https://doi.org/10.1016/j.tca.2022.179384)](https://doi.org/10.1016/j.tca.2022.179384)


### Conversion Modelling in FireSciPy
---------------------------------

In this introduction, the conversion of a single-step pyrolysis reaction is modelled. As an example, a linear heating rate of $\beta = 5~K/min$ is chosen for the temperature program. The parameters are taken from [Vyazovkin, Advanced Isoconversional Method, 1997 (https://doi.org/10.1007/BF01983708)](https://doi.org/10.1007/BF01983708).


In [ ]:

# Reaction constants
A = 10**10 / 60   # 1/s
E = 125.4 * 1000  # J/mol
alpha0 = 1e-12    # Avoid exactly zero to prevent numerical issues (e.g., division by zero)


At first, the linear temperature program needs to be created. Let's assume the sample temperature at the start of the experiment is at 300 K and it will end at a temperature of 750 K. The temperature is to change linearly over time according to the heating rate of $\beta = 5~K/min$. Heating rates can be provided in either $K/min$ or $K/s$, just pass the unit as a string, and FireSciPy will internally handle the unit conversion. The recording frequency of the TGA device is assumed to be such that a spacing of $\Delta T = 0.5~K$ is achieved. This spacing is a model of the frequency with which a TGA or similar device records the data.

In [ ]:

# Define temperature program (model recording frequency ΔT during TGA experiment)
n_points = 2*450 + 1    # ΔT = 0.5 K

# Temperatures in Kelvin
start_temp = 300
end_temp = 750


# Define model temperature program
beta = 5  # K/min


# Create the temperature program
temp_program = fsp.pyrolysis.modeling.create_linear_temp_program(
    start_temp=start_temp, 
    end_temp=end_temp, 
    beta=beta, 
    beta_unit="K/min", 
    steps=n_points)



Once the temperature program is set up, the conversion can be computed. The kinetics solver needs to be provided with a couple of parameters: the temperature program (`t_array`, `T_array`), the initial conversion level (`alpha0`), the Arrhenius parameters (`A`, `E`) and the reaction model.

FireSciPy comes with a small library of different reaction models like `'nth_order'` or `'Avrami_Erofeev'`. The values for their parameters can be provided as a dictionary.
In this example, we assume a first-order reaction model $f(\alpha) = (1 - \alpha)^n$ with $n~=~1$, which is commonly used in thermal decomposition modeling due to its simplicity.
Thus, the nth-order model is chosen (`'nth_order'`) with the order set to unity `{'n': 1.0}`. The result are data series of how the conversion (`alpha_sol`) changes over time (`t_sol`).

See also: `fsp.pyrolysis.modeling.create_linear_temp_program`, `fsp.pyrolysis.modeling.solve_kinetics`


In [ ]:

# Get time-temperature data series
time_model = temp_program["Time"]
temp_model = temp_program["Temperature"]

# Compute conversion
t_sol, alpha_sol = fsp.pyrolysis.modeling.solve_kinetics(
    t_array=time_model, 
    T_array=temp_model, 
    A=A, 
    E=E,
    alpha0=alpha0,
    R=fsp.constants.GAS_CONSTANT,
    reaction_model='nth_order',
    model_params={'n': 1.0})


Let's plot the conversion against the sample temperature to see the result. The code below will produce a plot showing the conversion as a function of temperature. Users are encouraged to run the example locally to see the result.

In [ ]:

# Plot conversion
plt.plot(temp_model, alpha_sol,
         label=f"{beta} K/min")


# Plot meta data
plt.title("Conversion of a Single-Step Pyrolyis Reaction")
plt.xlabel("Sample Temperature / K")
plt.ylabel("Conversion ($\\alpha$) / -")

plt.tight_layout()
plt.legend()
plt.grid()


For more details, see the [FireSciPy documentation](https://firedynamics.github.io/FireSciPy/).

## KAS Activation Energy Analysis from Simulated TGA

In this example, the conversion data is generated by leaveraging the modelling capabilities of FireSciPy. An example with experimental data is available too.

### Theoretical Background
----------------------

The conversion rate $\frac{d \alpha}{dt}$ of a single-step pyrolysis reaction can be described as presented in equation (4):

<!-- formula 4 -->
$$
\frac{d \alpha}{dt} = k(T) ~f(\alpha)
$$

and depends on the rate constant $k(T)$ and the reaction model $f(\alpha)$. Commonly, the rate constant is expressed as an Arrhenius equation (5), where $A$ is the pre-exponential factor, $E$ the activation energy and $R$ the gas constant.

<!-- formula 5 -->
$$
\frac{d \alpha}{dt} = A ~exp \left( -\frac{E}{R ~T}\right) ~f(\alpha)
$$

Together, $A$, $E$ and $f(\alpha)$ form the kinetic triplet.

Kinetics computations aim to deduce the elements of the kinetic triplet from experimental data. Here, specifically the determination of the activation energy $E$ is determined. It can be determined using an isoconversional method. These method are based on the observation that for a given level of conversion the reaction rate depends solely on the temperature. Taking the logarithmic derivative of equation (5) leads to equation (6). 

<!-- formula 6 -->
$$
\left[ \frac{\partial \ln \!\left(\dfrac{d\alpha}{dt}\right)}{\partial T^{-1}} \right]_{\alpha}
=
- \frac{E_\alpha}{R}
$$

At a constant conversion reaction model $f(\alpha)$ is constant and has no effect. For this reason isoconversional methods are sometimes called "model-free". However, this does not mean that $f(\alpha)$ can be ignored with respect to the kinetics triplet. 

For a constant heating rate program, the equation has no analytical solution and the temperature integral needs to be approximated. This is accomplished by recording data from experiments, for example thermogravimetric analysis (TGA). Data recorded at multiple different heating rates is used to determine the respective temperatures for a given level of conversion. The Kissinger–Akahira–Sunose (KAS) is a common isoconversional method, shown in equation (7), with $B=2$ and $C=1$. It has a higher accuracy compared to other well known methods like the Ozawa–Flynn–Wall (OFW) method. 

<!-- formula 7 -->
$$
\ln\!\left( \frac{\beta_i}{T_{\alpha,i}^{B}} \right) = \text{Const} - C \frac{E_{\alpha}}{R T_{\alpha}}
$$

The KAS method is further improved by Starink, setting $B=1.92$ and $C=1.0008$. The KAS method is available in FireSciPy and the Starink improvement is the default setup.


More details are available in the recommendations provided by the International Confederation for Thermal Analysis and Calorimetry (ICTAC) Kinetics Committee:

- [ICTAC Kinetics Committee recommendations for performing kinetic computations on thermal analysis data, 2011 (https://doi.org/10.1016/j.tca.2011.03.034)](https://doi.org/10.1016/j.tca.2011.03.034)
- [ICTAC Kinetics Committee recommendations for collecting experimental thermal analysis data for kinetic computations, 2014 (https://doi.org/10.1016/j.tca.2014.05.036)](https://doi.org/10.1016/j.tca.2014.05.036)
- [ICTAC Kinetics Committee recommendations for analysis of multi-step kinetics, 2020 (https://doi.org/10.1016/j.tca.2020.178597)](https://doi.org/10.1016/j.tca.2020.178597)
- [ICTAC Kinetics Committee recommendations for analysis of thermal decomposition kinetics, 2023 (https://doi.org/10.1016/j.tca.2022.179384)](https://doi.org/10.1016/j.tca.2022.179384)


### Synthetic TGA Data and KAS Estimation
-------------------------------------------------------------------------

FireSciPy provides functionalities to conduct reaction kinetics computations. In this example, the activation energy $E$ is determined, using the Kissinger-Akaira-Sunose (KAS) isoconversional method. 


In [ ]:

# Reaction constants
A = 10**10 / 60   # 1/s
E = 125.4 * 1000  # J/mol
alpha0 = 1e-12    # Avoid exactly zero to prevent numerical issues (e.g., division by zero)


FireSciPy introduces an internal data structure for the pyrolysis computations. The goal of this data structure is to keep all the data organised. Furthermore, the different methods of FireSciPy are aware of the structure and can thus easily call the data they need and store their own results. 

In this particular example, the reading and processing of experimental data is not discussed. Data series will be generated from modelling, therefore some steps are skipped. The generated data will be placed manually in the data structure in the locations where the processed experimental data would end up. A dedicated example with experimental data is available too.

Below, the skeleton of this data structure is initialised. It also allows to track some meta-data, like which material was investigated, what device was used or who performed the experiments. Furthermore, a dictionary with information on the recorded quantity (signal) is provided. This allows to address mass data from TGA, i.e. `{"name": "Mass", "unit": "mg"}`, or heat flow data from DSC, i.e. `{"name": "HeatFlow", "unit": "W/g"}`.


In [ ]:

# Initialise data structure for the kinetics assessment
data_structure = fsp.pyrolysis.kinetics.initialize_investigation_skeleton(
    material=f"Unobtainium", 
    investigator="John Doe, Miskatonic University", 
    instrument="ACME TGA 9000", 
    date="Stardate: 42.69", 
    notes="It has a colour out of space",
    signal={"name": "Mass", "unit": "mg"})


Basic settings for the temperature programs are defined below. For this example multiple heating rates are chosen and stored in a Python dictionary. The dictionary allows to easily keep track of temperature programs. 

Keep in mind, as per the ICTAC recommendations, the range of heating rates should cover at least a factor of 10 from the lowest to the highest. Furthermore, at the very least 3 heating rates should be used for isoconversional computations. Better would be 5 or more. Furthermore, basic isoconversional methods like KAS are built on the assumption that the heating rate is perfectly linear. This is captured here and the heating rates are thus labelled "nominal". 

The temperature programs all start at $300~K$ and end at $750~K$. Means are provided to adjust the sampling frequency (temperature spacing $\Delta T$). A few different frequencies are predefined and can easily be commented in to play around with.


In [ ]:

# Nominal heating rates
heating_rates = {
    "1Kmin": 1,
    "3Kmin": 3,
    "10Kmin": 10,
    "15Kmin": 15,
    "30Kmin": 30,
    "45Kmin": 45,
    "60Kmin": 60
}

# Data for temperature program (Kelvin)
T_start = 300
T_end = 750

# Set resolution for the temperature spacing (ΔT)
# resolution_factor = 0.5  # ΔT = 1.0 K
resolution_factor = 1  # ΔT = 0.5 K
# resolution_factor = 2  # ΔT = 0.25 K
# resolution_factor = 5  # ΔT = 0.1 K

# Number of points to be used in the arrays to be generated below
n_points = (int(900 * resolution_factor)) + 1


In the following code block, the individual temperature programs are created, based on the nominal heating rates. This is accomplished with the function `fsp.pyrolysis.modeling.create_linear_temp_program`. 

Looping over the above dictionary, a temperature program is created for each nominal heating rate. The time and temperature data series of each program are then extracted. They are used in the pyrolysis solver `fsp.pyrolysis.modeling.solve_kinetics`, together with the n-th order reaction model. The computed conversion ($\alpha$) is added to the temperature program DataFrame. The column headers of the DataFrame are renamed to match the expected values. Nominal values of the heating rates are store in the data structure, as well as the DataFrame. This information would typically come from the experiments and would then be added differently, see other example.


In [ ]:

# Modelling conversion and store in data structure
for hr_label in heating_rates:
    # Compute model heating rates
    beta = heating_rates[hr_label]
    hr_model = fsp.pyrolysis.modeling.create_linear_temp_program(
        start_temp=T_start, 
        end_temp=T_end, 
        beta=beta, 
        beta_unit="K/min", 
        steps=n_points)
    
    # Get temperature program
    time_model = fsp.utils.series_to_numpy(hr_model["Time"])
    temp_model = fsp.utils.series_to_numpy(hr_model["Temperature"])
    # Get overview over temperature resolution
    ΔT = temp_model[1] - temp_model[0]
    print(f"Temperature resolution ({hr_label}): ΔT = {ΔT} K")
    # Compute conversion for decelerating reaction (n-th order)
    t_sol, alpha_sol = fsp.pyrolysis.modeling.solve_kinetics(
        t_array=time_model, 
        T_array=temp_model, 
        A=A, 
        E=E,
        alpha0=alpha0,
        R=fsp.constants.GAS_CONSTANT,
        reaction_model='nth_order',
        model_params={'n': 1.0}
    )
    
    # Convert to DataFrame
    hr_model = pd.DataFrame(hr_model)
    # Add new column with conversion data
    hr_model["alpha"] = pd.Series(alpha_sol, index=hr_model.index)
    # Rename column headers to match expected values
    hr_model.rename(columns={
        'Time': 'Time', 
        'Temperature': 'Temperature_Avg', 
        'alpha': 'Alpha'}, 
                    inplace=True)
    
    # Ensure nested structure to store data, because model 
    # data is added manually here in this example and no data
    # processing from experimental TGA curves takes place
    keys = ["experiments", "TGA", "constant_heating_rate", hr_label]
    heating_rate = fsp.pyrolysis.kinetics.ensure_nested_dict(data_structure, keys)
    
    # Add nominal heating rates
    nominal_beta = {"value": beta, "unit": "K/min"}
    heating_rate["set_value"] = nominal_beta

    # Add the modeled conversion data to the data structure
    heating_rate["conversion"] = hr_model
    


Next, the desired conversion fractions need to be specified. They are necessary to ensure that the linear fit of the KAS method compares appropriate temperatures for a given level of conversion. During the experiments the data is recorded with a frequency that can be adjusted at the device. The changes that individual data points match up across many heating rates and for all desired conversion levels is very slim. Thus, interpolation of the input data is necessary. This interpolation is conducted with `fsp.pyrolysis.kinetics.compute_conversion_fractions`. As parameters, the data structure, the desired points for the analysis and the setup need to be provided. There is also an option to compute the fractions for selected temperature programs or all that are available.

Commonly, the conversion fractions range between (0.05 < $\alpha$ < 0.95) or (0.1 < $\alpha$ < 0.9). The reason being, that at the ends changes in are small over long times and the experimental noise is large. This leads to spurious results. However, the ends should not be rejected flat out. The user needs to assess how significant fluctuations are to not neglect, for example, initial reactions. In this example, a wider range is chosen: (0.01 < $\alpha$ < 0.99). Since the input data comes from modelling, the noise is low for the provided settings. It will be highlighted below that noise increases towards the end.


In [ ]:

# Define conversion fractions where to evaluate the activation energy
# Note: commonly, they range between (0.05 < α < 0.95) or (0.1 < α < 0.9)
# conversion_levels = np.linspace(0.05, 0.95, 37)  # Δα = 2.5
conversion_levels = np.linspace(0.01, 0.99, 99)  # Δα = 1.0

# Compute conversion fractions across all "experiments"
fsp.pyrolysis.kinetics.compute_conversion_levels(
    data_structure, 
    desired_levels=conversion_levels, 
    setup="constant_heating_rate", 
    condition="all")


The isoconversional activation energy can now be computed, using the KAS implementation `fsp.pyrolysis.kinetics.compute_Ea_KAS`. It simply needs to be provided with the data structure, all the necessary information should now be in the correct place.

By default, the KAS method implemented in FireSciPy uses the Starink improvement with the parameters $B=1.92$ and $C=1.0008$. For the classical KAS method, they can be changed to $B=2$ and $C=1$. See also: 
- [Starink, The determination of activation energy from linear heating rate experiments: a comparison of the accuracy of isoconversion methods, 2003 (https://doi.org/10.1016/S0040-6031(03)00144-8)](https://doi.org/10.1016/S0040-6031(03)00144-8)
- [ICTAC Kinetics Committee recommendations for performing kinetic computations on thermal analysis data, 2011 (https://doi.org/10.1016/j.tca.2011.03.034)](https://doi.org/10.1016/j.tca.2011.03.034)


In [ ]:

# Compute the activation energy using the KAS method
fsp.pyrolysis.kinetics.compute_Ea_KAS(data_structure, B=1.92, C=1.0008)


Finally, the results can be plotted: the development of the activation energy over the conversion. In gray, the first and last $5\%$ of the conversion are highlighted. Even using model data as input, it is observable that the noise increases towards the ends. Feel free to play with different sampling rates to see how they affect the result.


In [ ]:
fig, ax = plt.subplots()


# Get the Ea and convert to kJ/mol.
Ea_results_KAS = data_structure["experiments"]["TGA"]["Ea_results_KAS"]
Ea = Ea_results_KAS["Ea"]/1000
Ea_avg = np.average(Ea)

# Plot the Ea against conversion.
conv = Ea_results_KAS["Conversion"]
plt.scatter(conv,
            Ea,
            marker=".", s=42,
            facecolors="none",
            edgecolors="tab:blue",
            label=f"KAS, ΔT = {ΔT} K"
           )


# Plot target value, i.e. model input
plt.plot([0,1], [125.4, 125.4], 
         color="black", linestyle="--",
         label="Target, E$_a$=125.40 kJ/mol"
        )


# Shaded areas to indicate first/last 5 % to be typically cut off
x_min = -0.025
x_max = 1.025
ax.axvspan(x_min, 0.05, color='gray', alpha=0.3)
ax.axvspan(0.95, x_max, color='gray', alpha=0.3)


# Plot meta data.
plt.title(f"Activation Energy (KAS), E$_a$={Ea_avg:.2f} kJ/mol (avg.)")
plt.xlabel("Conversion ($\\alpha$) / -")
plt.ylabel("Activation Energy (E$_a$) / kJ/mol")

plt.xlim(left=x_min, right=x_max)
# plt.ylim(bottom=125.175, top=125.525)
plt.ylim(bottom=122.5, top=127.5)

plt.tight_layout()
plt.legend(loc="upper center")
plt.grid()


# # Save image.
# plot_label = f"Ea_Estimate_KAS.png"
# plot_path = os.path.join(plot_label)
# plt.savefig(plot_path, dpi=320, bbox_inches='tight', facecolor='w')


### Tracking of Intermediate Steps and Basic Statistics

FireSciPy keeps track of the intermediate steps during the kinetic analysis. Intermediate results are stored in the data structure and can be accessed to check the computation or share condensed results with others.

As an example, the conversion fractions for the different heating rates can easily be plottet as demonstrated below.


In [ ]:
# Pre-select data sets for convenience
heating_rate_data = data_structure["experiments"]["TGA"]["constant_heating_rate"]

# Go over all available heating rate data
for hr_label in heating_rate_data:
    # Get conversion fractions data series
    conv_frac_data = heating_rate_data[hr_label]["conversion_fractions"]
    temp = conv_frac_data["Temperature_Avg"]
    alpha = conv_frac_data["Alpha"]
    # Plot conversion against sample temperature
    plt.plot(temp, alpha, 
             linestyle='none', marker=".", ms=3,
             label=f"{hr_label[:-4]} K/min")


# Plot meta data
plt.title("Sampling Rate of Conversion Fractions")
plt.xlabel("Sample Temperature / K")
plt.ylabel("Conversion ($\\alpha$) / -")

plt.xlim(left=480, right=730)

plt.tight_layout()
plt.legend()
plt.grid()


# # Save image.
# plot_label = f"ConversionFractions_KAS.png"
# plot_path = os.path.join(plot_label)
# plt.savefig(plot_path, dpi=320, bbox_inches='tight', facecolor='w')


FireSciPy keeps also track of the individual fits conducted for the conversion faction, by the KAS method. This includes the parameters of the linear fit for each conversion level as well as the respecpective coordinates involved in the fit. These intermediate results are stored together with the primary results of the activation energy, as shown below.


In [ ]:
# Get the results of the activation energy
Ea_KAS = data_structure["experiments"]['TGA']["Ea_results_KAS"]

# Check results
Ea_KAS.head()


This information can be nicely used to visualise the KAS process. Below, data points and their respective fit are shown for a selection of conversion levels.

Note: Using the Starink improvement, the linear fit is taking place in the space of $ln\left(\frac{\beta}{T^{B}}\right)$ against $\frac{1}{T}$, with $B=1.92$. In case a different value for $B$ is chosen (see above) the space is adjusted accordingly. The user has to adjust the axis label accordingly.


In [ ]:
# Pre-select data sets for convenience
Ea_KAS = data_structure["experiments"]['TGA']["Ea_results_KAS"]

# Define settings for the plots of the fits
marker_size = 42
fit_line = ":"
fit_alpha = 0.6
fit_color = "black"

# Choose conversion levels for the plot
conversion_levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Initialise data collection
conv_temps = np.zeros((len(conversion_levels), len(heating_rates)))
conv_levels = np.zeros((len(conversion_levels), len(heating_rates)))

for conv_idx, level in enumerate(conversion_levels):
    level_idx = np.abs(Ea_KAS["Conversion"] - level).argmin()
    # Get the first three data points to match previous plot
    conv_temps[conv_idx,:] = np.asarray(Ea_KAS.loc[:,"x1":"x7"].iloc[level_idx])
    conv_levels[conv_idx,:] = np.asarray(Ea_KAS.loc[:,"y1":"y7"].iloc[level_idx])
    # Indicate fits
    m_fit = Ea_KAS["m_fit"].iloc[level_idx]
    b_fit = Ea_KAS["b_fit"].iloc[level_idx]
    x_fit = [conv_temps[conv_idx,:][0], conv_temps[conv_idx,:][-1]]
    y_fit = [fsp.utils.linear_model(x_fit[0], m_fit, b_fit),
             fsp.utils.linear_model(x_fit[-1], m_fit, b_fit)]
    
    if conv_idx == 0:
        plot_label = "Fit"
    else:
        plot_label = "_none_"
        
    plt.plot(x_fit, y_fit,
             linestyle=fit_line,
             alpha=fit_alpha,
             color=fit_color,
             label=plot_label)


hr_labels = list(heating_rates)
# Plot data points by heating rate
for idx in range(len(conv_temps.T)):
    # Get colour for data series, i.e. heating rate
    plot_colour = plt_colors[idx]
    
    hr_label = hr_labels[idx]
    # Plot data points
    plt.scatter(
        conv_temps.T[idx],
        conv_levels.T[idx],
        marker='o', s=marker_size,
        facecolors='none',
        edgecolors=plot_colour,
        label=f"{hr_label[:-4]} K/min")


# Plot meta data.
plt.title("Assess Linear Fits of KAS Method")
plt.xlabel("1/T")
plt.ylabel("ln($\\beta$/T$^{1.92}$)")

plt.xlim(left=0.00141, right=0.00197)
plt.ylim(bottom=-12.6, top=-7.9)

plt.tight_layout()
plt.legend()
plt.grid()

# # Save image.
# plot_label = f"Ea_Estimate_KAS_Fit.png"
# plot_path = os.path.join(plot_label)
# plt.savefig(plot_path, dpi=320, bbox_inches='tight', facecolor='w')


FireSciPy provides limited means of statistics to assess the quality of the linear regression fits during the KAS computation. This information is stored together with the primary and intermediate results of the activation energy computation.

Specifically, the coefficient of determination ($R^2$) and the root mean square error (RMSE) are available. They can be plotted together as follows:

In [ ]:
# Pre-select data sets for convenience
Ea_KAS = data_structure["experiments"]['TGA']["Ea_results_KAS"]

# Reduce number auf data points by using every n-th
nth = 1

markersize = 42

fig, ax = plt.subplots()

# Plot R² statistics
x_data = np.asarray(Ea_KAS["Conversion"])[::nth]
y_data = np.asarray(Ea_KAS["R_squared"])[::nth]
plt.scatter(x_data,
            y_data,
            marker='.', s=markersize,
            facecolors='none',
            edgecolors='tab:blue',
            label=f"R² avrg.: {np.average(y_data):.6f}")


# Plot RMSE statistics
y_data = np.asarray(Ea_KAS["RMSE"])[::nth]
plt.scatter(x_data,
            y_data,
            marker='.', s=markersize,
            facecolors='none',
            edgecolors='tab:orange',
            label=f"RMSE avrg.: {np.average(y_data):.6f}")


# Shaded areas to indicate first/last 5 % to be typically cut off
x_min = -0.025
x_max = 1.025
ax.axvspan(x_min, 0.05, color='gray', alpha=0.3, label="5%")
ax.axvspan(0.95, x_max, color='gray', alpha=0.3)


# Plot meta data.
plt.title("Statistics of the KAS Fits")
plt.xlabel("Conversion, $\\alpha$ / -")
plt.ylabel('Arbitrary Units / -')

plt.xlim(left=-0.025, right=1.025)
plt.ylim(bottom=-0.05, top=1.05)

plt.tight_layout()
plt.legend()
plt.grid()

# # Save image.
# plot_label = "Ea_Estimate_KAS_Statistics.png"
# plot_path = os.path.join(plot_label)
# plt.savefig(plot_path, dpi=320, bbox_inches='tight', facecolor='w')


For this example the results are a bit boring. Using experimental data the statistics are more useful. 

For more details, see the [FireSciPy documentation](https://firedynamics.github.io/FireSciPy/).


## Activation Energy Analysis (KAS) from TGA Experiments

In this example, the conversion data is derived from **experimental** thermogravimetric analysis (TGA) data.

### Theoretical Background
--------------------------

The apparent activation energy $E_a$ is estimated using the well-established isoconversional method known as Kissinger–Akahira–Sunose (KAS). For background on the theory and mathematical formulation, see the corresponding section in the [synthetic conversion data example](mock_kinetics_computation).

We make the distinction here, between the true activation energy $E$ and the apparent activation energy $E_a$. 

In micro-scale experiments such as TGA, it is generally not possible to deduce the fundamental reaction mechanisms or individual reaction steps involved in the decomposition. What appears as a single peak in the mass loss rate may in fact result from multiple overlapping reactions, with one step acting as rate-limiting and dominating the overall shape.

As such, the estimated activation energy from these experiments — $E_a$ — is considered apparent, reflecting the net behavior rather than a single elementary step.

The true activation energy is only definitively known in modeling scenarios, where the decomposition process is explicitly prescribed and controlled by the user (e.g., synthetic data or known reaction schemes).

See the ICTAC recommendations below for a detailed discussion of this distinction.

To apply the KAS method meaningfully, experimental data must be recorded under **multiple temperature programs**, either with **linear heating rates** or under **isothermal conditions**. The ICTAC Kinetics Committee recommends using at least three, preferably five, distinct datasets. If linear heating is used, the lowest and highest heating rates should differ by at least a factor of 10.

To ensure high-quality experimental data, a few key aspects must be considered:

- **Thermophysical effects of the sample material**: These influence the heat transfer within the sample and can distort the measured reaction rate. Their impact can be minimized by reducing the sample mass and repeating experiments with decreasing amounts until the results (e.g., mass loss rate curves) can be superimposed. This is especially important in micro-scale TGA experiments.

- **Secondary reactions involving evolved gases**: In some materials, volatile products may react with the remaining solid. This effect can be mitigated by increasing the purge gas flow rate to carry away evolved gases more effectively. Again, comparing mass loss rate curves for different flow rates can help confirm when the flow is sufficient (i.e., when the curves overlap).

In general, **apparent kinetic parameters should not be estimated from a single temperature program**. Multiple kinetic triplets can describe the same conversion curve equally well, making the solution **ambiguous**.

See also:
- [Single heating rate methods are a faulty approach to pyrolysis kinetics (https://doi.org/10.1007/s13399-022-03735-z)](https://doi.org/10.1007/s13399-022-03735-z)
- [Computational aspects of kinetic analysis.: Part B: The ICTAC Kinetics Project — the decomposition kinetics of calcium carbonate revisited, or some tips on survival in the kinetic minefield (https://doi.org/10.1016/S0040-6031(00)00444-5)](https://doi.org/10.1016/S0040-6031(00)00444-5)

More detailed guidance is available from the ICTAC Kinetics Committee:

- [ICTAC Kinetics Committee recommendations for performing kinetic computations on thermal analysis data, 2011 (https://doi.org/10.1016/j.tca.2011.03.034)](https://doi.org/10.1016/j.tca.2011.03.034)
- [ICTAC Kinetics Committee recommendations for collecting experimental thermal analysis data for kinetic computations, 2014 (https://doi.org/10.1016/j.tca.2014.05.036)](https://doi.org/10.1016/j.tca.2014.05.036)
- [ICTAC Kinetics Committee recommendations for analysis of multi-step kinetics, 2020 (https://doi.org/10.1016/j.tca.2020.178597)](https://doi.org/10.1016/j.tca.2020.178597)
- [ICTAC Kinetics Committee recommendations for analysis of thermal decomposition kinetics, 2023 (https://doi.org/10.1016/j.tca.2022.179384)](https://doi.org/10.1016/j.tca.2022.179384)

### Experimental TGA Data and KAS Estimation
--------------------------------------------

In this example, the apparent activation energy $E_a$ is estimated using the KAS method based on experimental TGA data. The data was recorded for a **sample mass of 1 mg**, using multiple linear heating rates.

Other datasets from the same study — including those with different sample masses and complementary microscale calorimeter data — are available at:

- [Thermogravimetric And Microscale Calorimeter Data on Cast PMMA (https://doi.org/10.24355/dbbs.084-202504170956-0)](https://doi.org/10.24355/dbbs.084-202504170956-0)


In this example we analyse TGA data recorded at different linear heating rates. For each heating rate, three repetitions were conducted. The data is provided in CSV format, one for each repetition and experimental condition. These are plain text files with a `.txt` extension, but they follow a typical tab-delimited CSV structure.

As in the synthetic data example, a Python dictionary is used to store all data related to this investigation. This structure is initialised using the function `initialize_investigation_skeleton`. 

Each CSV file is read into a Pandas DataFrame. If the files have meaningful names they can be directly used to label the data. For example, a file named `tga_dynamic_n2_dyn10_powder_1mg_r1.txt` corresponds to a mass of 1mg for a powdered sample, subjected to a 10 K/min heating rate and repetition 1.

The temperatures in the files are reported in degrees Celsius and must be converted to Kelvin before further processing, as FireSciPy's kinetic computations assume Kelvin. 

Once converted, each DataFrame is added to the data structure using `add_constant_heating_rate_tga`. This function stores the input data under the appropriate labels for **heating rate** and **repetition**.

Below is the code to read the files, adjust the temperatures, and store the results:




In [ ]:
# Path to the experimental data csv files.
fsp_data_path = os.path.join("C:\\", "path", "to", "the", "CSV_files")
exp_root = os.path.join(fsp_data_path, "docs", "tutorials", "pyrolysis", "data")


# Initialise data structure for the kinetics assessment
PMMA_data_1mg = fsp.pyrolysis.kinetics.initialize_investigation_skeleton(
    material=f"PMMA", 
    investigator="John Doe, Miskatonic University", 
    instrument="TGA/DSC 3+, Mettler Toledo", 
    date="Stardate: 42.69", 
    notes="Constant heating rates, sample mass 1mg",
    signal={"name": "Mass", "unit": "mg"})


for file_name in os.listdir(exp_root):
    if "powder_1mg_" in file_name:
        # print(file_name)
        
        # Parse metadata from file name: heating rate and repetition
        name_parts = file_name.split("_")
        hr_value = int(name_parts[3][3:])  # e.g., extracts 10 from 'dyn10'
        hr_label = f"{hr_value}_Kmin"
        print(hr_label)
        rep_label = f"Rep_{name_parts[-1][1]}"  # e.g., 'Rep_1' from '..._r1.txt'
        
        # Read CSV file as Pandas DataFrame
        exp_path = os.path.join(exp_root, file_name)
        exp_df = pd.read_csv(exp_path, header=0, skiprows=[1], 
                             delimiter=('\t'), encoding="cp858")
        
        # Adjust temperature to Kelvin
        exp_df["ts"] = exp_df["ts"] + 273.15
        exp_df["tr"] = exp_df["tr"] + 273.15
        
        # Add DataFrame to database
        fsp.pyrolysis.kinetics.add_constant_heating_rate_tga(
            database=PMMA_data_1mg, 
            condition=hr_label, 
            repetition=rep_label, 
            raw_data=exp_df, 
            data_type="integral", 
            set_value=[hr_value, "K/min"])
        
        
# Adjust column mapping for later functions 
column_mapping = {
        'time': 't',
        'temp': 'ts',
        'signal': 'weight'}
for hr_label in PMMA_data_1mg["experiments"]["TGA"]["constant_heating_rate"]:
    # Compute averages and standard deviations per heating rate
    fsp.pyrolysis.kinetics.combine_repetitions(
        database=PMMA_data_1mg, 
        condition=hr_label, 
        temp_program="constant_heating_rate",
        column_mapping=column_mapping)



Next, the conversion curves are computed using the function :func:`compute_conversion`. This function operates on the averaged data produced by :func:`combine_repetitions` in the previous step.

After this step, the computed conversion data will be available in the data structure and can be visualized or used in the isoconversional analysis.

To compute conversions for all available heating rates, simply run:


In [ ]:

fsp.pyrolysis.kinetics.compute_conversion(
    database=PMMA_data_1mg, 
    condition="all", 
    setup="constant_heating_rate")    
    


Now, the desired conversion levels must be specified to indicate where the apparent activation energy $E_a$ should be evaluated. 

Commonly, these levels lie in the range $0.10 < \alpha < 0.90$ or $0.05 < \alpha < 0.95$, depending on the quality of the experimental data. In the early and late stages of the reaction, the signal changes more slowly, and even small fluctuations or noise can lead to significant artifacts in the computed activation energy. This is typically visible in the $E_a$ versus conversion plots (see below).

However, these regions should not be discarded blindly. It's important to visually inspect the data to decide whether and to what extent the tails of the conversion curve can be included in the analysis.

The desired conversion levels can be provided as a NumPy array. The function `compute_conversion_levels` performs a linear interpolation of the available data to these levels in preparation for estimating the apparent activation energy.


In [ ]:

# Define conversion fractions where to evaluate the activation energy
# Note: commonly, they range between (0.05 < α < 0.95) or (0.1 < α < 0.9)
# conversion_levels = np.linspace(0.05, 0.95, 37)  # Δα = 2.5
conversion_levels = np.linspace(0.01, 0.99, 99)  # Δα = 1.0


fsp.pyrolysis.kinetics.compute_conversion_levels(
    database=PMMA_data_1mg, 
    desired_levels=conversion_levels,
    setup="constant_heating_rate", 
    condition="all")




The function `compute_Ea_KAS` performs the final step of the isoconversional method. Internally, it first sorts the available data by heating rate (from lowest to highest) for each conversion level. Then, for each level, it carries out a linear regression according to the KAS method (with the user-defined parameters $B$ and $C$).

The estimated activation energy values $E_a$ are stored in the database, alongside the intermediate fit data and statistics.

This step concludes the KAS-based estimation of the apparent activation energy.



In [ ]:

# Compute the activation energy using the KAS method
fsp.pyrolysis.kinetics.compute_Ea_KAS(
    database=PMMA_data_1mg, 
    B=1.92, 
    C=1.0008)



Below the result is plotted: the apparent activation energy against the conversion. In gray, the 5 % range at the ends is indicated.

The $E_a$ is computed in J/mol and converted here to kJ/mol, as it is a commen way to use it. 



Below, the apparent activation energy $E_a$ is plotted against the conversion $\alpha$.

The KAS method estimates $E_a$ in units of J/mol, but it is commonly reported in kJ/mol — so the values are converted accordingly before plotting.

The shaded gray regions indicate the first and last 5% of the conversion range. These are typically excluded from analysis due to higher sensitivity to noise and lower data reliability. In the first 5%, artifacts are visible due to increased noise.


In [ ]:
fig, ax = plt.subplots()


# Get the Ea and convert to kJ/mol.
Ea_results_KAS = PMMA_data_1mg["experiments"]["TGA"]["Ea_results_KAS"]
Ea = Ea_results_KAS["Ea"]/1000
Ea_avg = np.average(Ea)

# Plot the Ea against conversion.
conv = Ea_results_KAS["Conversion"]
plt.scatter(conv,
            Ea,
            marker=".", s=42,
            facecolors="none",
            edgecolors="tab:blue",
            label=f"KAS, ΔT = {ΔT} K")


# Shaded areas to indicate first/last 5 % (typically excluded)
x_min = -0.025
x_max = 1.025
ax.axvspan(x_min, 0.05, color='gray', alpha=0.3, label="5%")
ax.axvspan(0.95, x_max, color='gray', alpha=0.3)


# Plot meta data.
plt.title(f"Activation Energy (KAS), E$_a$={Ea_avg:.2f} kJ/mol (avg.)")
plt.xlabel("Conversion ($\\alpha$) / -")
plt.ylabel("Activation Energy (E$_a$) / kJ/mol")

plt.xlim(left=x_min, right=x_max)
plt.ylim(bottom=68, top=257)

plt.tight_layout()
plt.legend(loc="lower center")
plt.grid()


# # Save image.
# plot_label = f"Ea_Estimate_KAS.png"
# plot_path = os.path.join(plot_label)
# plt.savefig(plot_path, dpi=320, bbox_inches='tight', facecolor='w')



FireSciPy also evaluates the quality of each linear fit used in the KAS computation. Specifically, the **root mean square error (RMSE)** and the **coefficient of determination (R²)** are calculated.

- An **RMSE** of 0 indicates a perfect fit.
- An **R²** of 1 means that the fit perfectly explains the variance in the data.

In the plot below, fluctuations are more pronounced at low levels of conversion. This further supports the common practice of excluding the edges of the conversion range (typically the first and last 5%) in isoconversional analyses.


In [ ]:
# Pre-select data sets for convenience
Ea_KAS = PMMA_data_1mg["experiments"]['TGA']["Ea_results_KAS"]

# Optional: Reduce number of data points by using every n-th
nth = 1

markersize = 42

fig, ax = plt.subplots()

# Plot R² statistics
x_data = np.asarray(Ea_KAS["Conversion"])[::nth]
y_data = np.asarray(Ea_KAS["R_squared"])[::nth]
plt.scatter(x_data,
            y_data,
            marker='.', s=markersize,
            facecolors='none',
            edgecolors='tab:blue',
            label=f"R² avrg.: {np.average(y_data):.6f}")


# Plot RMSE statistics
y_data = np.asarray(Ea_KAS["RMSE"])[::nth]
plt.scatter(x_data,
            y_data,
            marker='.', s=markersize,
            facecolors='none',
            edgecolors='tab:orange',
            label=f"RMSE avrg.: {np.average(y_data):.6f}")


# Shaded areas to indicate first/last 5 % (typically excluded)
x_min = -0.025
x_max = 1.025
ax.axvspan(x_min, 0.05, color='gray', alpha=0.3, label="5%")
ax.axvspan(0.95, x_max, color='gray', alpha=0.3)


# Plot meta data.
plt.title("Statistics of the KAS Fits")
plt.xlabel("Conversion, $\\alpha$ / -")
plt.ylabel('Arbitrary Units / -')

plt.xlim(left=-0.025, right=1.025)
plt.ylim(bottom=-0.05, top=1.05)

plt.tight_layout()
plt.legend()
plt.grid()

# # Save image.
# plot_label = "Ea_Estimate_KAS_Statistics.png"
# plot_path = os.path.join(plot_label)
# plt.savefig(plot_path, dpi=320, bbox_inches='tight', facecolor='w')



Data produced during intermediate steps can also be accessed. For example, the averaged TG curves from the combined data sets can be used to plot normalised mass loss rates (MLR). This is shown below for heating rates of 30 K/min and 60 K/min.

The curves are aligned such that the final mass approaches zero, which is common when no significant residue remains. This zeroing helps compare curves consistently even when initial/final absolute values vary slightly. The MLR is then normalised by the initial mass to allow comparison between different heating rates.


In [ ]:
hr_labels = ["30_Kmin", "60_Kmin"]

for hr_label in hr_labels:
    # Access combined data
    PMMA_exp = PMMA_data_1mg["experiments"]["TGA"]["constant_heating_rate"][hr_label]["combined"]

    # Compute normalised mass loss rate
    time = PMMA_exp["Time"]
    temp_avg = PMMA_exp["Temperature_Avg"]
    mass_avg = PMMA_exp["Mass_Avg"]

    mlr = -np.gradient(mass_avg, time, edge_order=2)
    mlr_smooth = uniform_filter1d(mlr, size=9)
    
    # Plot the mass loss rate
    plt.plot(temp_avg, 
             mlr_smooth,
             label=f"{hr_label.split('_')[0]} K/min (1mg)")


# Plot meta data.
plt.xlabel("Sample Temperature / K")
plt.ylabel("Normalised Mass Loss Rate / 1/s")

plt.xlim(left=380,right=770)
plt.ylim(bottom=-0.0005,top=0.0165)

plt.legend()
plt.grid()


# Save image.
plot_label = f"NormalisedMLR.png"
plot_path = os.path.join(plot_label)
plt.savefig(plot_path, dpi=320, bbox_inches='tight', facecolor='w')


For more details, see the [FireSciPy documentation](https://firedynamics.github.io/FireSciPy/).


### Sensitivity of $E_a$ Estimation to the Number and Range of Heating Rates
-----------------------------------------------------------------------------

When estimating the apparent activation energy $E_a$, one common question is:  
**"Why are so many heating rates necessary? Can’t we just use three?"**

The answer to this question boils down to the fact that the isoconversional methods are fundamentally built on linear regression. For each chosen conversion level, the KAS method fits a straight line to $ln⁡(\beta/T^B)$ vs. $1/T$. The slope of this line is used to determine the apparent activation energy $E_a$.

According to the [ICTAC Kinetics Committee recommendations (https://doi.org/10.1016/j.tca.2014.05.036)](https://doi.org/10.1016/j.tca.2014.05.036), at the very least three, better are **five or more**, temperature programs should be used. These should span as wide a range as possible. Consider linear heating rates. ICTAC suggests to use a spread of a **factor of 10 or more** between the lowest and highest rate (e.g. 5 K/min to 50 K/min) to ensure robust estimation of $E_a$. This ensures:
- A wide spread in $1/T$ values, which increases statistical leverage in the fit.
- Enough data points to reduce sensitivity to noise and improve the robustness of the slope estimate.

If the heating rates are too close together or too few in number, the $1/T$ values cluster. This reduces the fit’s ability to capture the underlying trend, making the slope — and thus $E_a$ — more sensitive to experimental noise and measurement uncertainty.


In [ ]:

# Initialise data structure for the kinetics assessment
PMMA_data_1mg_low = fsp.pyrolysis.kinetics.initialize_investigation_skeleton(
    material=f"PMMA", 
    investigator="John Doe, Miskatonic University", 
    instrument="TGA/DSC 3+, Mettler Toledo", 
    date="Stardate: 42.69", 
    notes="Constant heating rates, sample mass 1mg",
    signal={"name": "Mass", "unit": "mg"})


# heating rates: 5, 10 K/min
file_names = [
    "tga_dynamic_n2_dyn5_powder_1mg_r1.txt",
    "tga_dynamic_n2_dyn5_powder_1mg_r2.txt",
    "tga_dynamic_n2_dyn5_powder_1mg_r3.txt",
    "tga_dynamic_n2_dyn10_powder_1mg_r1.txt",
    "tga_dynamic_n2_dyn10_powder_1mg_r2.txt",
    "tga_dynamic_n2_dyn10_powder_1mg_r3.txt"
]


for file_name in file_names:
    if "powder_1mg_" in file_name:
        # print(file_name)
        
        # Parse metadata from file name: heating rate and repetition
        name_parts = file_name.split("_")
        hr_value = int(name_parts[3][3:])  # e.g., extracts 10 from 'dyn10'
        hr_label = f"{hr_value}_Kmin"
        print(hr_label)
        rep_label = f"Rep_{name_parts[-1][1]}"  # e.g., 'Rep_1' from '..._r1.txt'
        
        # Read CSV file as Pandas DataFrame
        exp_path = os.path.join(exp_root, file_name)
        exp_df = pd.read_csv(exp_path, header=0, skiprows=[1], 
                             delimiter=('\t'), encoding="cp858")
        
        # Adjust temperature to Kelvin
        exp_df["ts"] = exp_df["ts"] + 273.15
        exp_df["tr"] = exp_df["tr"] + 273.15
        
        # Add DataFrame to database
        fsp.pyrolysis.kinetics.add_constant_heating_rate_tga(
            database=PMMA_data_1mg_low, 
            condition=hr_label, 
            repetition=rep_label, 
            raw_data=exp_df, 
            data_type="integral", 
            set_value=[hr_value, "K/min"])
        
        
# Adjust column mapping for later functions 
column_mapping = {
        'time': 't',
        'temp': 'ts',
        'signal': 'weight'}
for hr_label in PMMA_data_1mg_low["experiments"]["TGA"]["constant_heating_rate"]:
    # Compute averages and standard deviations per heating rate
    fsp.pyrolysis.kinetics.combine_repetitions(
        database=PMMA_data_1mg_low, 
        condition=hr_label, 
        temp_program="constant_heating_rate",
        column_mapping=column_mapping)
    

fsp.pyrolysis.kinetics.compute_conversion(
    database=PMMA_data_1mg_low, 
    condition="all", 
    setup="constant_heating_rate")   
    

# Define conversion fractions where to evaluate the activation energy
# Note: commonly, they range between (0.05 < α < 0.95) or (0.1 < α < 0.9)
# conversion_levels = np.linspace(0.05, 0.95, 37)  # Δα = 2.5
conversion_levels = np.linspace(0.01, 0.99, 99)  # Δα = 1.0

fsp.pyrolysis.kinetics.compute_conversion_levels(
    database=PMMA_data_1mg_low, 
    desired_levels=conversion_levels,
    setup="constant_heating_rate", 
    condition="all")


# Compute the activation energy using the KAS method
fsp.pyrolysis.kinetics.compute_Ea_KAS(
    database=PMMA_data_1mg_low, 
    B=1.92, 
    C=1.0008)




In [ ]:
# Pre-select data sets for convenience
Ea_KAS = PMMA_data_1mg["experiments"]['TGA']["Ea_results_KAS"]
Ea_KAS_example = PMMA_data_1mg_low["experiments"]['TGA']["Ea_results_KAS"]
hr_labels = ["5_Kmin", "10_Kmin", "20_Kmin", "30_Kmin", "60_Kmin"]

# Define settings for the plots of the fits
marker_size = 42
fit_line = ":"
fit_alpha = 0.8
fit_color = "black"

# Choose conversion levels for the plot
conversion_levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Initialise data collection
conv_temps = np.zeros((len(conversion_levels), len(hr_labels)))
conv_levels = np.zeros((len(conversion_levels), len(hr_labels)))

for conv_idx, level in enumerate(conversion_levels):
    level_idx = np.abs(Ea_KAS["Conversion"] - level).argmin()
    # Get the first three data points to match previous plot
    conv_temps[conv_idx,:] = np.asarray(Ea_KAS.loc[:,"x1":"x5"].iloc[level_idx])
    conv_levels[conv_idx,:] = np.asarray(Ea_KAS.loc[:,"y1":"y5"].iloc[level_idx])
    # Indicate fits
    m_fit = Ea_KAS["m_fit"].iloc[level_idx]
    b_fit = Ea_KAS["b_fit"].iloc[level_idx]
    x_fit = [conv_temps[conv_idx,:][0], conv_temps[conv_idx,:][-1]]
    y_fit = [fsp.utils.linear_model(x_fit[0], m_fit, b_fit),
             fsp.utils.linear_model(x_fit[-1], m_fit, b_fit)]
    
    if conv_idx == 0:
        plot_label = "Fit (full)"
    else:
        plot_label = "_none_"
    plt.plot(x_fit, y_fit,
             linestyle=fit_line,
             alpha=fit_alpha,
             color=fit_color,
             label=plot_label)
    
    # Indicate fits, extreme example
    m_fit = Ea_KAS_example["m_fit"].iloc[level_idx]
    b_fit = Ea_KAS_example["b_fit"].iloc[level_idx]
    
    y_fit = [fsp.utils.linear_model(x_fit[0], m_fit, b_fit),
             fsp.utils.linear_model(x_fit[-1], m_fit, b_fit)]
    
    if conv_idx == 0:
        plot_label = "Fit (5, 10)"
    else:
        plot_label = "_none_"
    plt.plot(x_fit, y_fit,
             linestyle=fit_line,
             alpha=fit_alpha,
             color="tab:red",
             label=plot_label)

# Plot data points by heating rate
for idx in range(len(conv_temps.T)):
    # Get colour for data series, i.e. heating rate
    plot_colour = plt_colors[idx]
    
    # Plot data points
    plt.scatter(
        conv_temps.T[idx],
        conv_levels.T[idx],
        marker='o', s=marker_size,
        facecolors='none',
        edgecolors=plot_colour,
        label=f"{hr_labels[idx].split('_')[0]} K/min")


# Plot meta data.
plt.title("Wide Heating Rate Ranges Improve Slope Stability in KAS")
plt.xlabel("1/T")
plt.ylabel("ln($\\beta$/T$^{1.92}$)")

plt.xlim(left=0.00141, right=0.00186)
plt.ylim(bottom=-11.1, top=-7.9)

plt.tight_layout()
plt.legend()
plt.grid()

# # Save image.
# plot_label = f"Ea_Estimate_KAS_Fit_PMMA1mg_example.png"
# plot_path = os.path.join(plot_label)
# plt.savefig(plot_path, dpi=320, bbox_inches='tight', facecolor='w')



The figure above illustrates this effect for a few conversion levels.     
- Black dotted lines: fits using the full dataset (five heating rates from 5 to 60 K/min).
- Red dotted lines: fits using only the two lowest heating rates (5 and 10 K/min).

With the narrower range, the points lie closer together along the $1/T$ axis, and the slope differs noticeably from the full-data case. This illustrates why **both range and redundancy** are critical when designing thermal analysis experiments for kinetic modeling.
